In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math as Math
from google.colab.patches import cv2_imshow # cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [23]:
pathOrigin = '/content/drive/My Drive/processamento-de-video/R6/'

É conhecido que os "corners", sendo os cantos ou vértices, são regiões da imagem com larga variação de intensidade em todas as direções.

Dessa forma, estudaremos a detecção dos "corners" a partir do modelo de Chris Harris, via a função `cornerHarris()` da biblioteca OpenCV, a qual leva o nome de seu criador em sua nomenclatura.

Execução:

Segundo a documentação, a imagem de input utilizada na função "should be grayscale and float32 type.", logo realizaremos as seguintes ações:

1. Leitura da imagem passando a flag `IMREAD_GRAYSCALE`, convertendo-a para grayscale
2. Transformando a imagem para o tipo `float32` via bibloteca NumPy

Para fins de testes, foram propostos duas maneiras de ler a imagem em `grayscale`, uma utilizando a flag no `imread()` e outra no `cvtColor()`, pois a documentação indica que há diferença.

> "When using IMREAD_GRAYSCALE, the codec's internal grayscale conversion will be used, if available. Results may differ to the output of cvtColor()"

Entretanto, não houveram diferenças visíveis no resultado da detecção dos vértices

In [37]:
# Função utilitária para transformar a imagem para "grayscale"
def grayMethod(method, fileName):
  # Caminho completo da imagem
  pathImage = pathOrigin + fileName
  if method == 1 :
    # Passando a flag cv.IMREAD_GRAYSCALE para a função imread
    return cv.imread(pathImage, cv.IMREAD_GRAYSCALE)
  elif method == 2 :
    image = cv.imread(pathImage)
    # Passando a flag cv.COLOR_BGR2GRAY para a função cvtColor
    return cv.cvtColor(image,cv.COLOR_BGR2GRAY)

In [38]:
def harrisCornerDetect(method, fileName):
  imageGray = grayMethod(method, fileName)
  # Transformando para tipo "float32"
  imageInModel = np.float32(imageGray)
  # Execução da função cornerHarris com os parâmetros indicados na documentação
  corners = cv.cornerHarris(imageInModel,2,3,0.04)
  # Dilatação do resultado para marcação dos corners
  corners = cv.dilate(corners,None)
  
  # Leitura da imagem original
  pathImage = pathOrigin + fileName
  image = cv.imread(pathImage)
  # Adição dos corners
  image[corners>0.01*corners.max()]=[0,0,255]
  cv2_imshow(image)
 

harrisCornerDetect(1, 'corner-image.jpg')
harrisCornerDetect(1, 'gitlab-logo.jpg')

harrisCornerDetect(2, 'corner-image.jpg')
harrisCornerDetect(2, 'gitlab-logo.jpg')

Output hidden; open in https://colab.research.google.com to view.

Alguns anos após a metodologia proposta por Chris Harris, J. Shi e C. Tomasi propuseram uma pequena modificação no código, na qual é definido um "threshold" e caso o valor do píxel seja maior que o threshold, o mesmo será considerado um "corner".

Execução:

Diferentemente da metodologia de Harris, segundo a documentação não é necessário que a imagem utilizada seja do tipo float32, porém, como usual, a imagem deve estar em grayscale.

Dessa forma foram realizadas as seguintes ações:

1. Leitura da imagem passando a flag `IMREAD_GRAYSCALE`, convertendo-a para grayscale
2. Utilizada a função proposta por Shi-Tomasi
3. Executada a função int0 da biblioteca Numpy para indexar os corners em números inteiros
4. Adicionado, via função cv.circle(), círculos nos N corners mais relevantes, indicados pelo valor "corners" na chamada da função `shiTomasiCornerDetect()`


Novamente foram testadas duas maneiras para leitura da imagem em 

1.   Item da lista
2.   Item da lista

"GrayScale", e novamente não houveram diferenças notáveis.

In [39]:
def shiTomasiCornerDetect(method, fileName, corners, qualityLevel):
  # Imagem para GrayScale
  imageGray = grayMethod(method, fileName)
  # Execução da função goodFeaturesToTrack 
  corners = cv.goodFeaturesToTrack(imageGray,corners,qualityLevel,10)
  corners = np.int0(corners)
  # Leitura da imagem original
  pathImage = pathOrigin + fileName
  image = cv.imread(pathImage)
  # Adição dos corners como círculos, com a função cv.circle, na imagem original
  for i in corners:
    x,y = i.ravel()
    cv.circle(image,(x,y),3,255,-1)
  # Mostra a imagem
  cv2_imshow(image)

 

#shiTomasiCornerDetect(1, 'corner-image.jpeg', 5, 0.5)
#shiTomasiCornerDetect(1, 'gitlab-logo.jpg', 5, 0.5)

shiTomasiCornerDetect(2, 'corner-image.jpg', 5, 0.1)
shiTomasiCornerDetect(2, 'gitlab-logo.jpg', 5, 0.1)
shiTomasiCornerDetect(2, 'corner-image.jpg', 10, 0.1)
shiTomasiCornerDetect(2, 'gitlab-logo.jpg', 10, 0.1)


shiTomasiCornerDetect(2, 'corner-image.jpg', 5, 0.5)
shiTomasiCornerDetect(2, 'gitlab-logo.jpg', 5, 0.5)
shiTomasiCornerDetect(2, 'corner-image.jpg', 10, 0.5)
shiTomasiCornerDetect(2, 'gitlab-logo.jpg', 10, 0.5)




Output hidden; open in https://colab.research.google.com to view.

In [17]:
def showImg(img):
  # Faz a leitura do objeto e exibe a imagem em tela, por padrão exibe em BGR,
  # assim como em "imread"
  cv2_imshow(img)


SIFT (Scale Invariant Feature Transform), é um algoritmo de visão computacional publicado por David Lowe. Ele é composto por duas partes distintas, que são: o detector e o descritor.

O detector SIFT é baseado em cálculos de diferença de Gaussianas e o descritor
SIFT utiliza histogramas de gradientes orientados para descrever a vizinhança
local dos pontos de interesse.

Execução:

1. Leitura da imagem passando a flag `IMREAD_GRAYSCALE`, convertendo-a para grayscale
2. Criação de um objeto SIFT, utilizando a função `SIFT_create` do openCV, porposta pelo tutorial.
3. Obtenção dos pontos-chave atravez do metodo `detectAndCompute`, do objroto SIFT criado do passo anterior, passando como parametro a imagem em grayscale, obtida no passo 1.
4. Usar a função `drawKeypoints`do openCV para desenhar os pontos-chave, ela recebe respectivament os parametro: a imagem em grayscale, os pontos-chave (obtidos no passo anterior), imagem original, pro fim é passada a flag `DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS`.

In [42]:
def keyPointsDetect(fileName):

  pathImage = pathOrigin + fileName
  image = cv.imread(pathImage)

  # Imagem para GrayScale
  imageGray= cv.cvtColor(image,cv.COLOR_BGR2GRAY)

  # Cria o objeto SIFT para tratar a imagem
  sift = cv.SIFT_create()

  # Executa a função detectAndCompute do objeto SIFT, que retorna os pontos-chave
  # e os descritores
  kp, des = sift.detectAndCompute(imageGray, None)

  # Desenha os pontos-chave na imagem
  img = cv.drawKeypoints(imageGray, kp, image, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

  # Exibe a imagem
  showImg(imageGray)


keyPointsDetect('corner-image.jpg')
keyPointsDetect('gitlab-logo.jpg')
keyPointsDetect('ufabc.jpeg')



AttributeError: ignored

A função `SIFT_create` do openCV não estava funcionando no colab, então a obtenção das imagens foi feito localmente usando o mesmo código acima, as imagens obtidas são exibidas abaixo.




In [36]:
def openImage(fileName):
  pathImage = pathOrigin + fileName
  image = cv.imread(pathImage)
  showImg(image)


openImage('corner-image_sift_keypoints.jpg')
openImage('gitlab_sift_keypoints.jpg')
openImage('ufabc_sift_keypoints.jpg')


Output hidden; open in https://colab.research.google.com to view.